# Tutorial 3. The Sellar Problem

Now that we have ran a simple calculator model using WISDEM, let's look at OpenMDAO. [OpenMDAO](http://openmdao.org) is the code that connects the various components of turbine models into a cohesive whole that can be optimized in systems engineering problems. WISDEM uses OpenMDAO to build up modular *components* and *groups* of components to represent a wind turbine.  Fortunately, OpenMDAO already provides some excellenet training examples on their [website](http://openmdao.org).  This tutorial is based on the OpenMDAO example, [Sellar - A Two-Discipline Problem with a Nonlinear Solver](http://openmdao.org/twodocs/versions/latest/basic_guide/sellar.html), which we have extracted and added some additional commentary. The aim of this tutorial is to summarize the key points you'll use to create basic WISDEM models.  For those interested in WISDEM development, getting comfortable with all of the core OpenMDAO training examples is strongly encouraged.

## Problem formulation

The Sellar problem are a couple of components (what Wikipedia calls models) that are simple equations. There is an objective to optimize and a couple of constraints to follow.

![Sellar XDSM](img/sellar_xdsm.png)

This is an XDSM diagram that is used to describe the problem and optimization setups. For more reference on this notation and general reference for multidisciplinary design analysis and optimization (MDAO), see:

- [Problem formulation section of multidisciplinary design optimization on Wikipedia](https://en.wikipedia.org/wiki/Multidisciplinary_design_optimization#Problem_formulation): Read the definitions for *design variables*, *constraints*, *objectives* and *models*.

- [Lambe and Martins: Extensions to the Design Strcuture Matrix for the Description of Multidisciplinary Desgn, Analysis, and Optimation Processes](http://mdolab.engin.umich.edu/content/extensions-design-structure-matrix): Read section 2 "Terminology and Notation" for further explanation of *design variables*, *discipline analysis*, *response variables*, *target variables* and *coupling variables*. Read section 4 about XDSM diagrams that dsecribe MDO processes.

## OpenMDAO implementation

First we need to import OpenMDAO

In [8]:
import openmdao.api as om
import numpy as np

Let's build *Discipline 1* first. On the XDSM diagram, notice the parallelograms connected to *Discipline 1* by thick grey lines. These are variables pertaining to the  *Discipline 1* component.

- \\(\mathbf{z}\\): An input. Since the components \\(z_1, z_2\\) can form a vector, we call the variable `z` in the code and initialize it to \\((0, 0)\\) with `np.zeros(2)`. Note that components of \\(\mathbf{z}\\) are found in 3 of the white \\(\mathbf{z}\\) parallelograms connected to multiple components and the objective, so this is a globabl design variable.

- \\(x\\): An input. A local design variable for Discipline 1. Since it isn't a vector, we just initialize it as a float.

- \\(y_2\\): An input. This is a coupling variable coming from an output on *Discipline 2*

- \\(y_1\\): An output. This is acoupling variable going to an input on *Discipline 2*

Let's take a look at the *Discipline 1* component and break it down piece by piece.
### Discipline 1

In [9]:
class SellarDis1(om.ExplicitComponent):
    """
    Component containing Discipline 1 -- no derivatives version.
    """

    def setup(self):

        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Local Design Variable
        self.add_input('x', val=0.)

        # Coupling parameter
        self.add_input('y2', val=1.0)

        # Coupling output
        self.add_output('y1', val=1.0)

        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')


    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y1 = z1**2 + z2 + x1 - 0.2*y2
        """
        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        x1 = inputs['x']
        y2 = inputs['y2']

        outputs['y1'] = z1**2 + z2 + x1 - 0.2*y2

The class declaration, `class SellarDis1(om.ExplicitComponent):` shows that our class, `SellarDis1` inherits off of the `ExplicitComponent` class in OpenMDAO.  In WISDEM, 99% of all coded components are of the `ExplicitComponent` class, so this is the most fundamental building block to get accustomed to.  Keen observers will notice that the *Sellar Problem* has implicitly defined variables that will need to be addressed, but that is addressed below.  Other types of components are described in the OpenMDAO docs [here](http://openmdao.org/twodocs/versions/latest/_srcdocs/packages/openmdao.components.html).

The `ExplicitComponent` class provides a template for the user to: 
- Declare their input and output variables in the `setup` method
- Calculate the outputs from the inputs in the `compute` method.  In an optimization loop, this is called at every iteration.
- Calculate analytical gradients of outputs with respect to inputs in the `compute_partials` method.  This is absent from the *Sellar Problem*.

The variable declarations take the form of `self.add_input` or `self.add_output` where a variable name and default/initial vaue is assigned.  The value declaration also tells the OpenMDAO internals about the size and shape for any vector or multi-dimensional variables.  Other optional keywords that can help with code documentation and model consistency are `units=` and `desc=`.

Finally `self.declare_partials('*', '*', method='fd')` tell OpenMDAO to use finite difference to compute the partial derivative of the outputs with respect to the inputs.  OpenMDAO provides many finite difference capabilities including:
- Forward and backward differencing
- Central differencing for second-order accurate derivatives
- Differencing in the complex domain which can offer improved accuracy for the models that support it

Now lets take a look at *Discipline 2*.

- \\(\mathbf{z}\\): An input comprised of \\(z_1, z_2\\).
- \\(y_2\\): An output. This is a coupling variable going to an input on *Discipline 1*
- \\(y_1\\): An input. This is a coupling variable coming from an output on *Discipline 1*

### Discipline 2

In [10]:
class SellarDis2(om.ExplicitComponent):
    """
    Component containing Discipline 2 -- no derivatives version.
    """

    def setup(self):
        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Coupling parameter
        self.add_input('y1', val=1.0)

        # Coupling output
        self.add_output('y2', val=1.0)

        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')


    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        y2 = y1**(.5) + z1 + z2
        """

        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        y1 = inputs['y1']

        outputs['y2'] = y1**.5 + z1 + z2

In OpenMDAO, multiple components can be connected together inside of a Group.  There will be some other new elements to review, so let's take a look:

### Sellar Group:

In [16]:
class SellarMDA(om.Group):
    """
    Group containing the Sellar MDA.
    """

    def setup(self):
        indeps = self.add_subsystem('indeps', om.IndepVarComp(), promotes=['*'])
        indeps.add_output('x', 1.0)
        indeps.add_output('z', np.array([5.0, 2.0]))

        self.add_subsystem('d1', SellarDis1(), promotes=['y1', 'y2'])
        self.add_subsystem('d2', SellarDis2(), promotes=['y1', 'y2'])
        self.connect('x', 'd1.x')
        self.connect('z', ['d1.z','d2.z'])

        # Nonlinear Block Gauss Seidel is a gradient free solver to handle implicit loops
        self.nonlinear_solver = om.NonlinearBlockGS()

        self.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                                  z=np.array([0.0, 0.0]), x=0.0),
                           promotes=['x', 'z', 'y1', 'y2', 'obj'])

        self.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
        self.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

The `SellarMDA` class derives off of the OpenMDAO `Group` class, which is typically the top-level class that is used in an analysis.  The OpenMDAO `Group` class allows you to cluster models in hierarchies. We can put multiple components in groups. We can also put other groups in groups.

Components are added to groups with the `self.add_subsystem` command, which has two primary arguments.  The first is the string name to call the subsystem that is added and the second is the component or sub-group class instance.  A common optional argument is `promotes=`, which elevatest the input/ouput variable string names to the top-level namespace.  The `SellarMDA` group shows examples where the `promotes=` can be passed a list of variable string names or the `'*'` wildcard to mean all input/output variables.

The first subsystem that is added is an `IndepVarComp`, which are the independent variables of the problem.  Subsystem inputs that are not tied to other subsystem outputs should be connected to an independent variables.  For optimization problems, design variables must be part of an `IndepVarComp`.  In the Sellar problem, we have `x` and `z`.  Note that they are promoted to the top level namespace, otherwise we would have to access them by `'indeps.x'` and `'indeps.z'`.

The next subsystems that are added are instances of the components we created above:
```python
self.add_subsystem('d1', SellarDis1(), promotes=['y1', 'y2'])
self.add_subsystem('d2', SellarDis2(), promotes=['y1', 'y2'])
```

The `promotes=` can also serve to connect variables.  In OpenMDAO, two variables with the same string name in the same namespace are automatically connected.  By promoting `y1` and `y2` in both `d1` and `d2`, they are automatically connected.  For variables that are not connected in this way, explicit connect statements are required such as:
```python
self.connect('x', ['d1.x','d2.x'])
self.connect('z', ['d1.z','d2.z'])
```
These statements connect the `IndepVarComp` versions of `x` and `z` to the `d1` and `d2` versions.  Note that if `x` and `z` could easily have been promoted in `d1` and `d2` too, which would have made these connect statements unnecessary, but including them is instructive.

The next statement, `self.nonlinear_solver = om.NonlinearBlockGS()`, handles the required internal iteration between `y1` and `y2` is our two components.  OpenMDAO is able to identify a *cycle* between input/output variables and requires the user to specify a solver to handle the nested iteration loop.  WISDEM does its best to avoid cycles.

Finally, we have a series of three subsystems that use instances of the OpenMDAO `ExecComp` component.  This is a useful way to defining an `ExplicitComponent` inline, without having to create a whole new class.  OpenMDAO is able to parse the string expression and populate the `setup` and `compute` methods automatically.  This technique is used to create our objective function and two constraint functions directly:
```python
self.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                          z=np.array([0.0, 0.0]), x=0.0),
                   promotes=['x', 'z', 'y1', 'y2', 'obj'])
self.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
self.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])
```
## Let's optimize our system!

Even though we have all the pieces in a `Group`, we still need to put them into a `Problem` to be executed.  The `Problem` instance is where we can assign design variables, objective functions, and constraints.  It is also how the user interacts with the `Group` to set initial conditions and interrogate output values.

First, we instantiate the `Problem` and assign an instance of `SellarMDA` to be the root model:

In [17]:
prob = om.Problem()
prob.model = SellarMDA()

Next we assign an optimization `driver` to the problem instance.  If we only wanted to evaluate the model once and not optimize, then a `driver` is not needed:

In [18]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-8

With the optimization driver in place, we can assign design variables, objective(s), and constraints.  Any `IndepVarComp` can be a design variable and any model output can be an objective or constraint.

In [19]:
prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

Now we are ready for to ask OpenMDAO to setup the model, to use finite differences for gradient approximations, and to run the driver:

In [20]:
prob.setup()

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.run_driver()

print('minimum found at')
print(prob['x'][0])
print(prob['z'])

print('minumum objective')
print(prob['obj'][0])

NL: NLBGS Converged in 7 iterations
NL: NLBGS Converged in 0 iterations
NL: NLBGS Converged in 3 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 8 iterations
NL: NLBGS Converged in 3 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 9 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 9 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 8 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 4 iterations
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.1833939